# Atelier 1 - Introduction à la variance de blocs

## 🎯 But pédagogique
Montrer comment la dépendance spatiale s'atténue avec la taille du support (**effet de support**).

---

### 🎓 Objectif

Montrer l’effet de la taille de support (bloc) sur la variance, comparée à :

- la **variance ponctuelle** ;
- la **variance de bloc théorique** (calculée à partir de l'intégrale de la fonction de covariance) ;
- la **variance de bloc expérimentale**, obtenue :
  - par **échantillonnage aléatoire** ;
  - ou par **moyenne sur sous-blocs**.

---

### 🔍 Concepts clés
- **Effet de support** : réduction de la variance lorsque la taille du bloc augmente.
- **Covariance spatiale** : mesure de la dépendance entre valeurs en fonction de la distance.
- **Variance de bloc** : mesure de la variabilité moyenne sur une surface ou un volume donné.

---



In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, Layout, IntSlider, FloatSlider, Dropdown, Output
from numpy.fft import fft2, ifft2, fftshift
from scipy.stats import norm

# --- Widgets avec style ---
slider_style = {'description_width': '160px'}
slider_layout = Layout(width='400px')

support_slider = IntSlider(min=0, max=50, step=1, value=1, description='Support (pixels)', style=slider_style, layout=slider_layout)
range_x_slider = FloatSlider(min=1, max=50, step=1, value=15, description='Portée X ($a_x$)', style=slider_style, layout=slider_layout)
range_y_slider = FloatSlider(min=1, max=50, step=1, value=15, description='Portée Y ($a_y$)', style=slider_style, layout=slider_layout)
sill_slider = FloatSlider(min=0.1, max=10, step=0.1, value=1.0, description='$c_1$', style=slider_style, layout=slider_layout)
nugget_slider = FloatSlider(min=0, max=1, step=0.01, value=0, description='Effet de pépite ($c_0$)', style=slider_style, layout=slider_layout)
angle_slider = FloatSlider(min=0, max=180, step=1, value=30, description='Angle ($θ$)', style=slider_style, layout=slider_layout)
model_dropdown = Dropdown(options=['Sphérique', 'Exponentiel', 'Gaussien'], value='Sphérique', description='Modèle', style=slider_style, layout=slider_layout)

# --- Bouton pour lancer le calcul ---
button = widgets.Button(description="Calculer", button_style='success', layout=Layout(width='200px'))

# --- Zone d'affichage graphique ---
out = Output()

# --- Fonction d’affichage ---
def interactive_variance(support, range_x, range_y, sill, nugget, angle_deg, model_name):
    if support > 50: support = 50
    
    field = fftma_simulation(
        size=500,
        range_x=range_x,
        range_y=range_y,
        sill=sill,
        nugget=nugget,
        angle_deg=angle_deg,
        model_name=model_name,
        seed=4263
    )
    field = normal_score_transform(field).reshape(field.shape)*np.sqrt(sill+nugget)
    
    plt.figure(figsize=(14,6))

    agg = aggregate(field, support)

    variance = sill + nugget
    std_dev = np.sqrt(variance)
    vmin = norm.ppf(0.05, loc=0, scale=std_dev)
    vmax = norm.ppf(0.95, loc=0, scale=std_dev)

    plt.subplot(1,2,1)
    plt.imshow(agg, cmap='viridis', origin='lower', vmin=vmin, vmax=vmax)
    plt.title(f'Champ agrégé, support = {support}x{support}')
    plt.colorbar(label='Valeurs')
    plt.axis('off')

    max_support = 50
    bsizes, var_exp = variance_blocks(field, max_support)

    var_theo = [
        theoretical_block_variance(
            range_x=range_x, range_y=range_y, sill=sill,
            nugget=nugget, block_size=b, pixel_size=0.05*b,
            angle_deg=angle_deg, model_name=model_name
        )
        for b in bsizes
    ]

    plt.subplot(1,2,2)
    plt.plot(bsizes, var_exp, 'o-', label='Variance expérimentale')
    plt.plot(bsizes, var_theo, 's--', label='Variance théorique')
    plt.axvline(support, color='red', linestyle=':', label=f'Support sélectionné = {support}')
    plt.xlabel('Taille du support (pixels)')
    plt.ylabel('Variance')
    plt.title('Variance des blocs vs taille de support')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.xlim(0, 50)
    plt.ylim(0, sill + nugget)
    plt.tight_layout()
    plt.show()

# --- Gestionnaire du clic sur le bouton ---
def on_button_clicked(b):
    with out:
        out.clear_output(wait=True)
        interactive_variance(
            support_slider.value,
            range_x_slider.value,
            range_y_slider.value,
            sill_slider.value,
            nugget_slider.value,
            angle_slider.value,
            model_dropdown.value
        )

button.on_click(on_button_clicked)

# --- Affichage interface ---
ui = VBox([
    support_slider,
    range_x_slider,
    range_y_slider,
    sill_slider,
    nugget_slider,
    angle_slider,
    model_dropdown,
    button
])

display(ui, out)

Output()